## IMPORTS

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive
drive.mount('/content/gdrive')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Import necessary libraries
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
trainFolder = "gdrive/My Drive/MLDM - Carrefour Project/data-train"

import os
for dirname, _, filenames in os.walk(f'{trainFolder}'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Mounted at /content/gdrive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

gdrive/My Drive/MLDM - Carrefour Project/data-train/products_data.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/test_data.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_1.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_10.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_2.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_3.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_4.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_5.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_6.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_7.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_8.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_9.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/products_embe.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/p

In [2]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

### Data Description

This project uses three datasets:

#### `train_data.csv`:
This dataset contains two years (2022 & 2023) of historical transactions for 100,000 Carrefour customers. It has 10 columns:

* ***date***: Date of the transaction
* ***transaction_id***: ID of the transaction
* ***customer_id***: Customer ID
* ***product_id***: Product purchased
* ***has_loyality_card***: Flag indicating whether the customer has a loyalty card
* ***store_id***: Store where the purchase was made
* ***is_promo***: Flag indicating whether there was a discount on the product
* ***quantity***: Quantity purchased of the product
* ***format***: Ecommerce activity format (clcv, lex, or DRIVE)
  - clcv : courses livrées chez vous
  - lex : livraison express
  - DRIVE.
* ***orderChannelCode***: Indicates whether the online activity was made through the website or mobile app

#### `products_data.csv`:
This dataset contains detailed information about the products. The following columns are relevant to this project:

* ***product_id*** : Product name
* ***product_description*** : Product description
* ***department_key***: Department key
* ***class_key***: Class key
* ***subclass_key***: Subclass key
* ***sector***: sector name
* ***brand_key***: Brand name
* ***shelf_level1***: Top-level shelf category
* ***shelf_level2***: Second-level shelf category
* ***shelf_level3***: Third-level shelf category
* ***shelf_level4***: Fourth-level shelf category
* ***sector***: Sector
* ***bio***: Flag indicating whether the product is organic
* ***sugar_free***: Flag indicating whether the product is sugar-free
* ***aspartame_free***: Flag indicating whether the product is aspartame-free
* ***gluten_free***: Flag indicating whether the product is gluten-free
* ***halal***: Flag indicating whether the product is halal
* ***casher***: Flag indicating whether the product is kosher
* ***eco_friendly***: Flag indicating whether the product is eco-friendly
* ***local_french***: Flag indicating whether the product is locally produced in France
* ***artificial_coloring_free***: Flag indicating whether the product is free of artificial coloring
* ***taste_enhancer_free***: Flag indicating whether the product is free of taste enhancers
* ***naturality***: Naturality score
* ***antibiotic_free***: Flag indicating whether the product is antibiotic-free
* ***reduced_sugar***: Flag indicating whether the product has reduced sugar content
* ***vegetarian***: Flag indicating whether the product is vegetarian
* ***pesticide_free***: Flag indicating whether the product is pesticide-free
* ***grain_free***: Flag indicating whether the product is grain-free
* ***no_added_sugar***: Flag indicating whether the product has no added sugar
* ***salt_reduced***: Flag indicating whether the product has reduced salt content
* ***nitrite_free***: Flag indicating whether the product is nitrite-free
* ***fed_without_ogm***: Flag indicating whether the animals were fed without GMOs
* ***no_added_salt***: Flag indicating whether the product has no added salt
* ***no_artificial_flavours***: Flag indicating whether the product has no artificial flavors
* ***porc***: Flag indicating whether the product contains pork
* ***vegan***: Flag indicating whether the product is vegan
* ***frozen***: Flag indicating whether the product is frozen
* ***fat_free***: Flag indicating whether the product is fat-free
* ***reduced_fats***: Flag indicating whether the product has reduced fat content
* ***fresh***: Flag indicating whether the product is fresh
* ***alcool***: Flag indicating whether the product contains alcohol
* ***lactose_free***: Flag indicating whether the product is lactose-free
* ***phenylalanine_free***: Flag indicating whether the product is phenylalanine-free
* ***palm_oil_free***: Flag indicating whether the product is palm oil-free
* ***ecoscore***: Ecoscore
* ***produits_du_monde***: Flag indicating whether the product is an international product
* ***regional_product***: Flag indicating whether the product is a regional product
* ***national_brand***: Flag indicating whether the product is a national brand
* ***first_price_brand***: Flag indicating whether the product is a first-price brand
* ***carrefour_brand***: Flag indicating whether the product is a Carrefour brand

#### `test_data.csv`:
This dataset contains the actual purchases of the first 80,000 customers in 2024. It has three columns:

* ***transaction_id***: ID of the transaction
* ***customer_id***: Customer ID
* ***product_id***: the id of the purchased product

### Load data

* Load *train_data.csv*, *products_data.csv* and *test_data.csv* using pandas.

In [3]:
# This code reads the data from CSV files named "train_data_part_i.csv" for all i from 1 to 10
# and concatenates them into a single pandas DataFrame
train_dataframes = []
for i in tqdm(range(1, 6)): ## 1 - 11
    train_dataframes.append(pd.read_csv(f'{trainFolder}/train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

100%|██████████| 5/5 [01:11<00:00, 14.23s/it]


In [4]:
len(train_data)

43664701

In [5]:
#train_data = train_data.sample(frac=0.1, random_state=1)  # frac=0.1 takes 10% of the data

In [6]:
# This code reads the data from a CSV file named "products_data.csv" into a pandas DataFrame
products_data = pd.read_csv(f'{trainFolder}/products_data.csv')

<ipython-input-6-d8cc84ca7717>:2: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv(f'{trainFolder}/products_data.csv')


In [7]:
# This code reads the data from a CSV file named "test_data.csv" into a pandas DataFrame
test_data = pd.read_csv(f'{trainFolder}/test_data.csv')

In [8]:
products_data_emb = pd.read_csv(f'{trainFolder}/products_data_bert1.csv')

In [9]:
import re
import ast

# Function to fix leading spaces and format the embedding
def fix_embedding_format(embedding):
    if not isinstance(embedding, str) or not embedding.strip():
        return None  # Handle missing or null embeddings

    try:
        # Remove leading space, outer brackets, and format properly
        cleaned = embedding.strip()  # Remove leading and trailing spaces
        cleaned = cleaned.lstrip('[')  # Remove the first "[" without spaces
        cleaned = re.sub(r'^\s+', '', cleaned)  # Explicitly remove leading space after '['
        cleaned = cleaned.rstrip(']')  # Remove the final "]"
        formatted = re.sub(r'\s+', ',', cleaned)  # Replace all remaining spaces with commas
        return ast.literal_eval(f'[{formatted}]')  # Convert to a Python list
    except Exception as e:
        print(f"Error fixing embedding: {embedding}, Error: {e}")
        return None  # Return None for rows that fail parsing

# Apply the fix
products_data_emb['embedding'] = products_data_emb['embedding'].apply(fix_embedding_format)

# Verify the fix
print("Sample fixed embedding:", products_data_emb['embedding'].iloc[0])


Sample fixed embedding: [-0.885095, -0.381127149, 0.523640215, -0.206593141, -0.257077605, 0.112018563, -0.29885143, 0.959406734, -0.714935005, 0.129923865, 0.0166194402, -0.0755547956, -0.0377267376, 1.04182911, 0.163626552, 0.751397073, 0.0291998107, 0.551166594, -0.264074653, -0.399605095, -0.201198012, -0.874935985, 1.06481993, 0.0418045074, -0.0420345031, 0.272841483, 0.104067206, -0.250343502, -0.091578193, 0.319404036, -0.209970936, 0.5293383, -0.359875351, -0.21511282, 0.866470695, 0.288113505, 0.590503991, -0.146806508, -0.245605424, 0.263168812, -0.121223584, 0.356522411, 0.157572299, 0.0094260294, 0.379927129, 0.188013539, -3.55864835, -0.106123105, -0.664489448, -0.504262984, 0.303419292, -0.447763115, 0.0806763694, 0.379628479, 0.864170611, 0.570908725, -0.5299896, -0.0537453741, 0.321317106, 0.087104708, 0.538094044, -0.342478484, -0.116726674, 0.121116519, 0.44294259, -0.102612607, 0.337457687, 0.0163220558, -0.519234657, 0.820987165, -0.533936977, -0.325439185, 0.102716

In [10]:
print(products_data_emb['embedding'].head())
print(f"Number of invalid embeddings: {products_data_emb['embedding'].isnull().sum()}")

0    [-0.885095, -0.381127149, 0.523640215, -0.2065...
1    [-0.959080517, -0.365773737, 0.405761927, -0.2...
2    [-0.864090204, -0.386676431, 0.545451403, -0.1...
3    [-0.876440346, -0.245052218, 0.520699978, -0.1...
4    [-0.829412699, -0.389280558, 0.462360829, -0.1...
Name: embedding, dtype: object
Number of invalid embeddings: 0


In [11]:
products_data_emb.head()

,product_id,embedding
0,Product_33508,"[-0.885095, -0.381127149, 0.523640215, -0.2065..."
1,Product_15347,"[-0.959080517, -0.365773737, 0.405761927, -0.2..."
2,Product_80604,"[-0.864090204, -0.386676431, 0.545451403, -0.1..."
3,Product_81740,"[-0.876440346, -0.245052218, 0.520699978, -0.1..."
4,Product_82516,"[-0.829412699, -0.389280558, 0.462360829, -0.1..."


In [12]:
train_data.sample(5)

,date,transaction_id,customer_id,product_id,has_loyality_card,store_id,is_promo,quantity,format,order_channel
11174517,2023-03-28,Transaction_778289,Household_18181,Product_64296,0,Store_397,0,1.0,DRIVE,WEBSITE
1753323,2022-05-01,Transaction_563150,Household_5709,Product_51816,0,Store_1609,0,1.0,DRIVE,MOBILE_APP
32331693,2022-09-23,Transaction_2521396,Household_36924,Product_24400,0,Store_1083,0,1.0,DRIVE,WEBSITE
14484104,2022-04-28,Transaction_1054441,Household_11631,Product_56786,0,Store_1549,0,2.0,DRIVE,MOBILE_APP
8553015,2023-04-11,Transaction_2618574,Household_5647,Product_50099,0,Store_697,0,1.0,DRIVE,WEBSITE


In [13]:
products_data.sample(3)

,product_id,product_description,department_key,class_key,subclass_key,sector,brand_key,shelf_level1,shelf_level2,shelf_level3,shelf_level4,bio,sugar_free,aspartame_free,gluten_free,halal,casher,eco_friendly,local_french,artificial_coloring_free,taste_enhancer_free,naturality,antibiotic_free,reduced_sugar,vegetarian,pesticide_free,grain_free,no_added_sugar,salt_reduced,nitrite_free,fed_without_ogm,no_added_salt,no_artificial_flavours,porc,vegan,frozen,fat_free,reduced_fats,fresh,alcool,lactose_free,phenylalanine_free,palm_oil_free,ecoscore,produits_du_monde,regional_product,national_brand,first_price_brand,carrefour_brand
63527,Product_7824,PET 1L NECTAR ABRICOT CRF SEL,Department_10,Class_1001,SubClass_10013,PGC,CRF SELEC,Boissons,Jus de fruits et légumes,Jus de pomme et fruits du verger,Jus d'abricot,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C,0,0,0,0,1
15147,Product_45145,250G MLU BIO CARTE,Department_14,Class_1405,SubClass_14053,PGC,CARTE NOIR,Bio et Ecologie,Epicerie sucrée,"Thés, Cafés et Infusions",Cafés,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0
73304,Product_53309,360G FLT SAUM PANE CRF EXTRA,Department_25,Class_2583,SubClass_25830,PGC,CRF EXTRA,Surgelés,Poissons et Fruits de mer,Poissons panés,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,0,0,0,0,1


In [14]:
test_data.sample(3)

,transaction_id,customer_id,product_id
501514,Transaction_2024_53615,Household_18461,Product_65416
651091,Transaction_2024_38507,Household_47939,Product_52740
193794,Transaction_2024_47636,Household_18380,Product_30700


## Data Preproccesing

In [15]:
# Ensure 'date' is in datetime format
train_data['date'] = pd.to_datetime(train_data['date'])

# Extract temporal features
train_data['month'] = train_data['date'].dt.month
train_data['day_of_week'] = train_data['date'].dt.dayofweek
train_data['is_weekend'] = train_data['day_of_week'].isin([5, 6]).astype(int)

# Calculate recency: days since the last purchase for each customer
train_data['days_since_last_purchase'] = train_data.groupby('customer_id')['date'].transform(
    lambda x: (x.max() - x).dt.days
)

# Drop the 'date' column if it's no longer needed
train_data = train_data.drop(columns=['date'], errors='ignore')

In [16]:
# Aggregate customer-specific features
customer_features = train_data.groupby('customer_id').agg(
    purchase_frequency=('product_id', 'count'),  # Total number of purchases
    avg_quantity=('quantity', 'mean'),          # Average quantity purchased
    promo_ratio=('is_promo', 'mean'),           # Proportion of purchases made with promotions
    unique_products=('product_id', 'nunique')   # Number of unique products purchased
).reset_index()

# Merge these features into train_data and test_data
train_data = train_data.merge(customer_features, on='customer_id', how='left')

In [17]:
# Check for missing values
print("Missing values in train_data:")
print(train_data.isnull().sum())

print("Missing values in test_data:")
print(test_data.isnull().sum())

# Fill remaining missing values with zeros as a safety measure
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

Missing values in train_data:
transaction_id              0
customer_id                 0
product_id                  0
has_loyality_card           0
store_id                    0
is_promo                    0
quantity                    0
format                      0
order_channel               0
month                       0
day_of_week                 0
is_weekend                  0
days_since_last_purchase    0
purchase_frequency          0
avg_quantity                0
promo_ratio                 0
unique_products             0
dtype: int64
Missing values in test_data:
transaction_id    0
customer_id       0
product_id        0
dtype: int64


In [18]:
train_data.head()

,transaction_id,customer_id,product_id,has_loyality_card,store_id,is_promo,quantity,format,order_channel,month,day_of_week,is_weekend,days_since_last_purchase,purchase_frequency,avg_quantity,promo_ratio,unique_products
0,Transaction_1730125,Household_39,Product_5362,0,Store_2,0,1.0,DRIVE,MOBILE_APP,11,2,0,0,282,1.219858,0.014184,185
1,Transaction_1560535,Household_39,Product_67174,0,Store_2,0,2.0,DRIVE,WEBSITE,7,2,0,483,282,1.219858,0.014184,185
2,Transaction_1560535,Household_39,Product_82254,0,Store_2,0,2.0,DRIVE,WEBSITE,7,2,0,483,282,1.219858,0.014184,185
3,Transaction_1730125,Household_39,Product_3895,0,Store_2,0,1.0,DRIVE,MOBILE_APP,11,2,0,0,282,1.219858,0.014184,185
4,Transaction_1560535,Household_39,Product_34014,0,Store_2,0,1.0,DRIVE,WEBSITE,7,2,0,483,282,1.219858,0.014184,185


In [19]:
train_data = train_data.drop(columns=['has_loyality_card', 'store_id', 'day_of_week','is_weekend' ,'format','order_channel','month'])

In [20]:
train_data.head()

,transaction_id,customer_id,product_id,is_promo,quantity,days_since_last_purchase,purchase_frequency,avg_quantity,promo_ratio,unique_products
0,Transaction_1730125,Household_39,Product_5362,0,1.0,0,282,1.219858,0.014184,185
1,Transaction_1560535,Household_39,Product_67174,0,2.0,483,282,1.219858,0.014184,185
2,Transaction_1560535,Household_39,Product_82254,0,2.0,483,282,1.219858,0.014184,185
3,Transaction_1730125,Household_39,Product_3895,0,1.0,0,282,1.219858,0.014184,185
4,Transaction_1560535,Household_39,Product_34014,0,1.0,483,282,1.219858,0.014184,185


In [21]:
products_data_emb.head()

,product_id,embedding
0,Product_33508,"[-0.885095, -0.381127149, 0.523640215, -0.2065..."
1,Product_15347,"[-0.959080517, -0.365773737, 0.405761927, -0.2..."
2,Product_80604,"[-0.864090204, -0.386676431, 0.545451403, -0.1..."
3,Product_81740,"[-0.876440346, -0.245052218, 0.520699978, -0.1..."
4,Product_82516,"[-0.829412699, -0.389280558, 0.462360829, -0.1..."


In [22]:
train_data = train_data.merge(products_data_emb, on='product_id', how='left')
train_data.head()

,transaction_id,customer_id,product_id,is_promo,quantity,days_since_last_purchase,purchase_frequency,avg_quantity,promo_ratio,unique_products,embedding
0,Transaction_1730125,Household_39,Product_5362,0,1.0,0,282,1.219858,0.014184,185,"[-0.714311779, -0.163191468, 0.362559706, -0.2..."
1,Transaction_1560535,Household_39,Product_67174,0,2.0,483,282,1.219858,0.014184,185,"[-0.798357368, -0.396710098, 0.51669389, -0.09..."
2,Transaction_1560535,Household_39,Product_82254,0,2.0,483,282,1.219858,0.014184,185,"[-0.704730511, -0.268889457, 0.566852391, -0.0..."
3,Transaction_1730125,Household_39,Product_3895,0,1.0,0,282,1.219858,0.014184,185,"[-0.843781114, -0.310565889, 0.639390349, -0.2..."
4,Transaction_1560535,Household_39,Product_34014,0,1.0,483,282,1.219858,0.014184,185,"[-0.797278345, -0.273620039, 0.310368985, -0.2..."


## RNN


In [25]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence

# Preprocess customer data into a dictionary for faster access
customer_data_dict = {customer_id: data for customer_id, data in train_data.groupby('customer_id')}

# Create a dataset for sequential modeling
class PurchaseDataset(Dataset):
    def __init__(self, customer_data_dict):
        self.customer_data_dict = customer_data_dict
        self.customers = list(customer_data_dict.keys())

    def __len__(self):
        return len(self.customers)

    def __getitem__(self, idx):
        customer_id = self.customers[idx]
        customer_data = self.customer_data_dict[customer_id]
        sequence = torch.tensor(np.vstack(customer_data['embedding'].apply(lambda x: np.array(eval(x) if isinstance(x, str) else x)).values), dtype=torch.float32)
        label = int(customer_data['product_id'].values[-1].split('_')[-1])  # Extract numerical ID from 'product_id'
        return sequence, label

# Custom collate function to pad sequences
def collate_fn(batch):
    sequences, labels = zip(*batch)
    padded_sequences = pad_sequence(sequences, batch_first=True)
    labels = torch.tensor(labels, dtype=torch.long)
    return padded_sequences, labels

# Create DataLoader
train_dataset = PurchaseDataset(customer_data_dict)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

# Define LSTM Model
class PurchaseLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(PurchaseLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

# Hyperparameters
input_size = len(np.array(eval(train_data['embedding'].iloc[0]) if isinstance(train_data['embedding'].iloc[0], str) else train_data['embedding'].iloc[0]))
hidden_size = 128
output_size = products_data['product_id'].nunique()
num_layers = 1
learning_rate = 0.001
num_epochs = 4

# Model, Loss, Optimizer
model = PurchaseLSTM(input_size, hidden_size, output_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    model.train()
    correct = 0
    total = 0
    with tqdm(train_loader, unit="batch") as tepoch:
        for sequences, labels in tepoch:
            tepoch.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            tepoch.set_postfix(loss=loss.item())

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.2f}%')

# Evaluation with Hit Rate@10
def hit_rate_at_10(model, test_data, products_data):
    model.eval()
    hit_count = 0
    total_count = 0
    with tqdm(test_data['customer_id'].unique(), unit="customer") as tcustomer:
        for customer_id in tcustomer:
            if customer_id not in customer_data_dict:
                continue
            customer_history = customer_data_dict[customer_id]

            sequence = torch.tensor(np.vstack(customer_history['embedding'].apply(lambda x: np.array(eval(x) if isinstance(x, str) else x)).values), dtype=torch.float32).unsqueeze(0)

            with torch.no_grad():
                output = model(sequence)
                top_10_indices = torch.topk(output, 10).indices.squeeze().tolist()
                actual_product = int(test_data[test_data['customer_id'] == customer_id]['product_id'].values[0].split('_')[-1])

                if actual_product in top_10_indices:
                    hit_count += 1
                total_count += 1

    hit_rate = hit_count / total_count
    print(f'Hit Rate@10: {hit_rate:.4f}')

# Run evaluation
hit_rate_at_10(model, test_data, products_data)



Epoch [1/4]: 100%|██████████| 3125/3125 [1:28:29<00:00,  1.70s/batch, loss=9.15]


Epoch [1/4], Accuracy: 0.38%


Epoch [2/4]: 100%|██████████| 3125/3125 [1:30:22<00:00,  1.74s/batch, loss=8.42]


Epoch [2/4], Accuracy: 0.40%


Epoch [3/4]: 100%|██████████| 3125/3125 [1:30:40<00:00,  1.74s/batch, loss=8.12]


Epoch [3/4], Accuracy: 0.40%


Epoch [4/4]: 100%|██████████| 3125/3125 [1:29:49<00:00,  1.72s/batch, loss=8.79]


Epoch [4/4], Accuracy: 0.44%


100%|██████████| 80000/80000 [1:53:25<00:00, 11.76customer/s] 

Hit Rate@10: 0.0276


In [39]:
# Generate top 10 recommendations for a specific customer
def recommend_top_10(model, customer_id, customer_data_dict, products_data):
    model.eval()
    if customer_id not in customer_data_dict:
        print("No history available for this customer.")
        return None

    customer_history = customer_data_dict[customer_id]
    sequence = torch.tensor(np.vstack(customer_history['embedding'].apply(lambda x: np.array(eval(x) if isinstance(x, str) else x)).values), dtype=torch.float32).unsqueeze(0)

    with torch.no_grad():
        output = model(sequence)
        top_10 = torch.topk(output, 10)
        top_10_indices = top_10.indices.squeeze().tolist()
        top_10_scores = top_10.values.squeeze().tolist()
        recommended_products = [products_data['product_id'].iloc[i] for i in top_10_indices]
        ranks = list(range(1, 11))  # Rank from 1 to 10 based on the scores
        return {'customer_id': customer_id, 'recommended_products': recommended_products, 'rank': ranks, 'scores': top_10_scores}

# Generate recommendations for households between 80001 and 100000
recommendations = []
with tqdm(range(1001, 21001), unit="customer") as tcustomer:
    for i in tcustomer:
        result = recommend_top_10(model, f'Household_{i}', customer_data_dict, products_data)
        if result:
            recommendations.append(result)

# Create DataFrame with recommendations
recommendation_df = pd.DataFrame(recommendations)

# Keep only the top 10 recommendations for Households between 80001 and 100000
prediction = recommendation_df[recommendation_df['customer_id'].isin([f"Household_{i}" for i in range(1001, 21001)])]

# Print the solution
print(prediction.head())

100%|██████████| 20000/20000 [24:35<00:00, 13.55customer/s]

      customer_id                               recommended_products  \
0  Household_1001  [Product_56824, Product_3935, Product_28134, P...   
1  Household_1002  [Product_35666, Product_4190, Product_25618, P...   
2  Household_1003  [Product_25618, Product_35666, Product_4190, P...   
3  Household_1004  [Product_4190, Product_68649, Product_35203, P...   
4  Household_1005  [Product_3935, Product_56824, Product_64685, P...   

                              rank  \
0  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   
1  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   
2  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   
3  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   
4  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]   

                                              scores  
0  [1.262446641921997, 0.9793286919593811, 0.8494...  
1  [0.783717691898346, 0.6966193318367004, 0.6799...  
2  [0.9166085124015808, 0.8665592074394226, 0.798...  
3  [0.649742066860199, 0.472578763961792, 0.29136...  
4  [2.2092642784118652, 1.9726800918579102, 1.427...  


In [40]:
prediction.to_csv('gdrive/My Drive/MLDM - Carrefour Project/submission/submission_list_rnn.csv', index=False)

In [29]:
def process_and_format_prediction(df):
    # Remplacement des caractères invalides dans les noms de colonnes
    df.columns = df.columns.str.replace('+AF8-', '_', regex=False)
    df = df.replace(r'\+AF8-', '_', regex=True)

    # Nettoyage des colonnes 'customer_id', 'product_id', et 'transaction_id'
    if 'customer_id' in df.columns and df['customer_id'].dtype == 'object':
        df['customer_id'] = df['customer_id'].str.extract('(\d+)').fillna(11).astype(int)
    if 'product_id' in df.columns and df['product_id'].dtype == 'object':
        df['product_id'] = df['product_id'].str.extract('(\d+)').fillna(11).astype(int)
    if 'transaction_id' in df.columns and df['transaction_id'].dtype == 'object':
        df['transaction_id'] = df['transaction_id'].str.replace(r'\D', '', regex=True).fillna(11).astype(int)

    df['id'] = df.index
    df = df[['id'] + [col for col in df.columns if col != 'id']]

    if 'customer_id' not in df.columns or 'product_id' not in df.columns:
        raise ValueError("true_data must contain 'customer_id' and 'product_id' columns")

    # Grouper par customer_id et concaténer les valeurs des produits et des ranks
    prediction_grouped = df.groupby('customer_id').agg({
        'id': 'first',  # Prend la première valeur de 'id'
        'product_id': lambda x: ','.join(map(str, x)),  # Concatène les product_id en chaîne de caractères
        'rank': lambda x: ','.join(map(str, x))  # Concatène les ranks en chaîne de caractères
    }).reset_index()

    # Supprimer la colonne 'id' si elle existe
    if 'id' in prediction_grouped.columns:
        prediction_grouped = prediction_grouped.drop(columns=['id'])

    # Filtrer les données
    prediction_grouped = prediction_grouped[prediction_grouped['customer_id'] != 11]
    prediction_grouped.insert(0, 'id', range(len(prediction_grouped)))

       # Vérification des rangs et des doublons
    for index, row in prediction_grouped.iterrows():
        # Vérifier les ranks
        ranks = list(map(int, row['rank'].split(',')))
        if sorted(ranks) != list(range(1, 11)):  # Vérifie que les rangs sont distincts de 1 à 10
            print("Doublon détecté. Les rangs doivent être distincts (de 1 à 10) pour chacun des 10 produits prédits pour un client.\n")
            return None
        # Vérifier les doublons de produits
        products = row['product_id'].split(',')
        if len(products) != len(set(products)):  # Si des doublons sont présents dans les produits
            print("Doublon détecté. Il doit y avoir 10 produits différents par client.\n")
            return None


    return prediction_grouped
prediction_grouped=process_and_format_prediction(prediction)
print(prediction_grouped)


ValueError: true_data must contain 'customer_id' and 'product_id' columns

In [41]:
# Process and format the prediction dataframe
def process_and_format_prediction(df):
    # Explode the 'recommended_products' column to create individual product_id rows
    df = df.explode(['recommended_products', 'scores', 'rank'])
    df = df.rename(columns={'recommended_products': 'product_id'})
    df['product_id'] = df['product_id'].str.extract('(\d+)').fillna(11).astype(int)

    # Replacement of invalid characters in column names
    df.columns = df.columns.str.replace('+AF8-', '_', regex=False)
    df = df.replace(r'\+AF8-', '_', regex=True)

    # Cleaning the 'customer_id' column
    if 'customer_id' in df.columns and df['customer_id'].dtype == 'object':
        df['customer_id'] = df['customer_id'].str.extract('(\d+)').fillna(11).astype(int)

    df['id'] = df.index
    df = df[['id'] + [col for col in df.columns if col != 'id']]

    if 'customer_id' not in df.columns or 'product_id' not in df.columns:
        raise ValueError("prediction dataframe must contain 'customer_id' and 'product_id' columns")

    # Group by 'customer_id' and concatenate the product, rank, and score values
    prediction_grouped = df.groupby('customer_id').agg({
        'id': 'first',  # Take the first value of 'id'
        'product_id': lambda x: ','.join(map(str, x)),  # Concatenate product_id as a string
        'rank': lambda x: ','.join(map(str, x)),  # Concatenate rank as a string
        'scores': lambda x: ','.join(map(str, x))  # Concatenate scores as a string
    }).reset_index()

    # Drop the 'id' column if it exists
    if 'id' in prediction_grouped.columns:
        prediction_grouped = prediction_grouped.drop(columns=['id'])

    # Filter the data
    prediction_grouped = prediction_grouped[prediction_grouped['customer_id'] != 11]
    prediction_grouped.insert(0, 'id', range(len(prediction_grouped)))

    # Verification of ranks and duplicates
    for index, row in prediction_grouped.iterrows():
        # Verify ranks
        ranks = list(map(int, row['rank'].split(',')))
        if sorted(ranks) != list(range(1, 11)):  # Ensure ranks are distinct from 1 to 10
            print("Duplicate detected. Ranks must be distinct (from 1 to 10) for each of the 10 predicted products for a customer.\n")
            return None
        # Verify product duplicates
        products = row['product_id'].split(',')
        if len(products) != len(set(products)):  # If duplicates are present in products
            print("Duplicate detected. There must be 10 different products per customer.\n")
            return None

    return prediction_grouped

# Apply the function to process the prediction dataframe
prediction_grouped = process_and_format_prediction(prediction)
prediction_grouped.head()

<ipython-input-41-0985a99a71f6>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'\+AF8-', '_', regex=True)


,id,customer_id,product_id,rank,scores
0,0,1001,"56824,3935,28134,64685,38838,38929,23160,39962...","1,2,3,4,5,6,7,8,9,10","1.262446641921997,0.9793286919593811,0.8494096..."
1,1,1002,"35666,4190,25618,68649,35203,30700,76553,55012...","1,2,3,4,5,6,7,8,9,10","0.783717691898346,0.6966193318367004,0.6799982..."
2,2,1003,"25618,35666,4190,55012,35999,68649,5169,30700,...","1,2,3,4,5,6,7,8,9,10","0.9166085124015808,0.8665592074394226,0.798702..."
3,3,1004,"4190,68649,35203,36819,25618,7003,10714,38519,...","1,2,3,4,5,6,7,8,9,10","0.649742066860199,0.472578763961792,0.29136109..."
4,4,1005,"3935,56824,64685,28134,38929,23160,38838,61819...","1,2,3,4,5,6,7,8,9,10","2.2092642784118652,1.9726800918579102,1.427287..."


In [37]:
#prediction_grouped.to_csv('submission/submission_list_bert.csv', index=False) ####
prediction_grouped.to_csv('gdrive/My Drive/MLDM - Carrefour Project/submission/submission_list_rnn_r.csv', index=False)